# Limpando Dados após fazer um Scraping usando Scrapy

Foi realizado um processo de Scraping numa página de Imóveis da cidade de Porto Alegre. O processo foi feito utilizando a biblioteca Scrapy e ao término foi gerado um arquivo JSON com os dados que foram coletados.
Neste texto iremos analisar os dados que foram coletados e realizar a limpeza e manutenção.....

In [1]:
import pandas as pd
import numpy as np

import re

In [2]:
file = "datasets/imoveis.json"

df = pd.read_json(file, orient = 'records')

In [3]:
df.head()

,preco,dormitorios,area,vagas,bairro
0,[R$ 1.055.000],[3],[128],[3],"[Imóvel 106391 - , Apartamento, \n ..."
1,[R$ 1.112.000],[3],[136.34],[2],"[Imóvel 130624 - , Apartamento, \n ..."
2,[R$ 1.620.000],[3],[204.87],[3],"[Imóvel 192740 - , Apartamento, \n ..."
3,[R$ 1.190.000],[3],[241],[2],"[Imóvel 196741 - , Casa, \n ..."
4,[R$ 3.053.011],[4],[271.3],[4],"[Imóvel 183630 - , Apartamento, \n ..."


In [4]:
df.shape

(600, 5)

In [5]:
list(df.columns)

['preco', 'dormitorios', 'area', 'vagas', 'bairro']

In [6]:
df.dtypes

preco          object
dormitorios    object
area           object
vagas          object
bairro         object
dtype: object

Como vimos, as colunas 'preco', 'area' e 'vagas' estão com o tipo de dado 'object'. Por se tratarem de valores numéricos, iremos convertê-las para o tipo de dado numérico, possibilitando-nos futuramente tirarmos insights.

In [7]:
def cleaning_values(value):
    value = str(value)
    value = value.replace("Valor Previsto", "").replace("R$", "") \
                 .replace(".", "").replace("[", "").replace("]", "") \
                 .replace("'", "").replace(" ", "")
    # print(value)

    return value

In [8]:
def cleaning_values_bairro(value):
    value = str(value)
    value = value.replace(".", "").replace("[", "").replace("]", "") \
                 .replace("Imóvel", "").replace("-", "") \
                 .replace("'", "").replace("Porto Alegre", "") \
                 .replace('"', "").replace(" ", "")
    # print(value)

    return value

In [9]:
df_clean = df.copy()

In [10]:
df_clean['area'] = df_clean['area'].apply(cleaning_values).astype(float)

In [11]:
df_clean['preco'] = df_clean['preco'].apply(cleaning_values).astype(float)

In [12]:
df_clean['dormitorios'] = df_clean['dormitorios'].apply(cleaning_values).astype(float)

In [13]:
df_clean['vagas'] = df_clean['vagas'].apply(cleaning_values).astype(float)

In [14]:
df_clean['bairro'] = df_clean['bairro'].apply(cleaning_values_bairro)

In [15]:
df_clean['bairro'] = df_clean['bairro'].apply(lambda x : re.sub(r"\d", "", str(x)))

In [16]:
df_clean['tipo'] = df_clean['bairro'].apply(lambda x : re.split(',', x)[1])

In [17]:
df_clean['bairro'] = df_clean['bairro'].apply(lambda x : re.split(',', x)[2])

In [18]:
df_clean['bairro'] = df_clean['bairro'].apply(lambda x : re.sub(r'\\n', '', x))

In [19]:
df_clean['tipo'].head()

0    Apartamento
1    Apartamento
2    Apartamento
3           Casa
4    Apartamento
Name: tipo, dtype: object

In [20]:
df_clean['bairro'].head()

0    CentralParque
1       MeninoDeus
2        RioBranco
3          Sarandi
4        BelaVista
Name: bairro, dtype: object

In [21]:
df_clean.isnull().sum()

# FILLNA

preco           0
dormitorios     8
area            0
vagas          21
bairro          0
tipo            0
dtype: int64

In [22]:
df_clean.dtypes

preco          float64
dormitorios    float64
area           float64
vagas          float64
bairro          object
tipo            object
dtype: object

In [23]:
df_clean.isnull().sum().values.sum()

29

In [24]:
df_clean.describe()

,preco,dormitorios,area,vagas
count,6.000000e+02,592.000000,600.000000,579.000000
mean,8.636065e+05,2.584459,8829.695000,1.787565
std,9.296298e+05,0.764899,14787.265011,0.978020
min,1.178000e+05,1.000000,42.000000,1.000000
25%,3.800000e+05,2.000000,685.750000,1.000000
50%,5.870000e+05,3.000000,6530.000000,2.000000
75%,9.710585e+05,3.000000,10249.000000,2.000000
max,9.117117e+06,6.000000,229098.000000,8.000000


In [25]:
df_clean.groupby(by = 'area').preco.mean()

area
42.0         325000.0
45.0         440000.0
46.0         220000.0
48.0         219000.0
51.0         280000.0
              ...    
75151.0     6500000.0
98528.0      630000.0
99616.0      769000.0
136512.0    1100000.0
229098.0    1500000.0
Name: preco, Length: 490, dtype: float64